In [1]:
from load_data import load_data

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
from imblearn.over_sampling import SMOTE

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [112]:
import csv

In [86]:
import statistics

In [26]:
from glob import glob
import os
import pandas as pd
import numpy as np

In [5]:
def segmentation(x_data_list,y_list,overlap_rate,time_window):
    
    seg_data = []
    overlap = int((1 - overlap_rate)*time_window)
    y_segmented_list = []
    for x_data,y in zip(x_data_list,y_list):
        for i in range(0,x_data.shape[0],overlap):
            seg_data.append(x_data[i:i+time_window])
            y_segmented_list.append(y)
    
    return seg_data,y_segmented_list

In [38]:
def segmentation_with_seg_num(x_data_list,seg_id_list,overlap_rate,time_window):
    
    seg_data = []
    overlap = int((1 - overlap_rate)*time_window)
    seg_id_list_increase = []
    for x_data,seg_id in zip(x_data_list,seg_id_list):
        for i in range(0,x_data.shape[0],overlap):
            seg_data.append(x_data[i:i+time_window])
            seg_id_list_increase.append(seg_id)
    
    return seg_data,seg_id_list_increase

In [7]:
def get_features(x_data):
    features = []
    for i in range(x_data.shape[1]):
        # std
        features.append(x_data.T[i].std(ddof=0))
        # avg
        features.append(np.average(x_data.T[i]))
        # max
        features.append(np.max(x_data.T[i]))
        # min
        features.append(np.min(x_data.T[i]))
        # kurtosis
#         features.append(stats.kurtosis(x_data[0].T[i]))
    return features

In [57]:
def load_test_data(csv_folder_path):
    csv_file_paths = glob(csv_folder_path)
    x_data_list = []
    seg_id_list = []
    for csv_file in csv_file_paths:
        csv_df = pd.read_csv(csv_file)
        file_name = os.path.basename(csv_file)
        seg_id = os.path.splitext(file_name)[0].replace("segment","")
        x_data_list.append(csv_df.drop(["timestamp","Activity"],axis=1).values)
        seg_id_list.append(int(seg_id))

    return x_data_list,seg_id_list

In [18]:
X_user0001,y_user0001 = load_data("segdata/0001/*")
X_user0003,y_user0003 = load_data("segdata/0003/*")
X_user0002,y_user0002 = load_data("segdata/0002/*")

In [21]:
x_seg_data,y_seg_data = segmentation(X_user0001+X_user0002+X_user0003,y_user0001+y_user0002+y_user0003,overlap_rate=0.5,time_window=100)

In [22]:
model_ml = RandomForestClassifier(n_estimators=500,n_jobs=-1)

In [28]:
X_features_data = []

In [29]:
for train in x_seg_data:
    X_features_data.append(get_features(train))

In [30]:
print(len(x_seg_data))
print(len(y_seg_data))
model_ml.fit(X_features_data,y_seg_data)

41440
41440


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [58]:
x_data_list_test,seg_id_list = load_test_data("segdata_test/0004/*")

In [120]:
a = [len(x_data) for x_data in x_data_list_test]

In [121]:
print(sum(a))

437013


In [59]:
x_seg_data_test,seg_id_list_increase = segmentation_with_seg_num(x_data_list_test,seg_id_list,overlap_rate=0.5,time_window=100)

In [64]:
print(len(x_seg_data_test))
print(len(seg_id_list_increase))

8760
8760


In [65]:
target_features = []
# print((x_seg_data))
for target in x_seg_data_test:
    target_features.append(get_features(target))

In [73]:
print(len(target_features))
print(len(seg_id_list_increase))

target_features = np.array(target_features)

8760
8760


In [93]:
predict_dict = {}

In [95]:
def predict_testdata(clf,x_test,seg_id):
    y_predict = clf.predict(x_test)
#     print(y_predict)
    for y_,seg_id in zip(y_predict,seg_id_list_increase):
        predict_dict.setdefault(int(seg_id),[]).append(y_)
    return predict_dict

In [96]:
model_ml

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [97]:
predict_label_dict = predict_testdata(model_ml,target_features,seg_id_list_increase)

[7 7 3 ... 1 1 7]


In [117]:
for seg_id in predict_label_dict:
    pred = statistics.mode(predict_label_dict[seg_id])
    print(seg_id,pred)  
    timestamp_list = get_timestamp(seg_id)
    for timestamp in timestamp_list:
        write_csv([timestamp,pred])

3 3
2 8
1 2
5 1
4 3
6 3
7 3
34 5
20 7
21 8
35 5
23 3
22 2
26 2
32 5
33 2
27 7
19 1
31 4
25 8
24 2
30 8
18 8
15 3
29 8
28 3
14 8
16 3
17 1
13 3
12 7
10 2
11 4
9 8
8 1


In [111]:
def get_timestamp(seg_id):
    df_test_data = pd.read_csv("segdata_test/0004/segment{}.csv".format(seg_id))
    timestamp_list = df_test_data["timestamp"].values
    return timestamp_list

In [116]:
timestamp_list = get_timestamp(1)
pred = statistics.mode(predict_label_dict[1])
for timestamp in timestamp_list:
    write_csv([timestamp,pred])

In [109]:
print(len(timestamp_list))

12012


In [115]:
def write_csv(write_list):
    with open("result.csv",mode="a",encoding="utf-8") as f:
        writer = csv.writer(f,lineterminator='\n')
        writer.writerow(write_list)